In [1]:
import glob
import numpy as np
import pandas as pd
import os
import shutil
import matplotlib.pyplot as plt
import random
import keras
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from sklearn.preprocessing import LabelEncoder
from keras.applications import resnet50, vgg16
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer, Lambda, GlobalAveragePooling2D, BatchNormalization
from keras.models import Model, Sequential
from keras import optimizers
from keras import backend as K
from itertools import combinations
%matplotlib inline

Using TensorFlow backend.


In [2]:
# set to image directory
cov = glob.glob('/Users/austinau-yeung/Documents/Georgia Tech/1/ECE6254/project/covid-chestxray-dataset-master/output/*')
pne = glob.glob('/Users/austinau-yeung/Documents/Georgia Tech/1/ECE6254/project/pneumonia2/*')

# parameters
# cov_train_num = 400
# pne_train_num = 400
cov_train_num = 140
pne_train_num = 140

In [3]:
num_classes = 2

# select random subset of images for training
cov_train = np.random.choice(cov,size=cov_train_num,replace=False)
pne_train = np.random.choice(pne,size=pne_train_num,replace=False)
cov = list(set(cov)-set(cov_train))
pne = list(set(pne)-set(pne_train))
cov_test = cov
pne_test = pne

cov_test_num = len(cov_test)
pne_test_num = len(pne_test)

In [4]:
IMG_WIDTH = 224
IMG_HEIGHT = 224
IMG_DIM = (IMG_WIDTH,IMG_HEIGHT)

# load training images
cov_train_imgs = [img_to_array(load_img(img,target_size=IMG_DIM,color_mode="rgb")) for img in cov_train]
pne_train_imgs = [img_to_array(load_img(img,target_size=IMG_DIM,color_mode="rgb")) for img in pne_train]

# create corresponding labels
train_imgs = np.array(cov_train_imgs+pne_train_imgs)
train_imgs_scaled = train_imgs.astype('float32')/255
train_labels = cov_train_num*['c']+pne_train_num*['p']

# load test images and create corresponding labels
cov_test_imgs = [img_to_array(load_img(img,target_size=IMG_DIM,color_mode="rgb")) for img in cov_test]
pne_test_imgs = [img_to_array(load_img(img,target_size=IMG_DIM,color_mode="rgb")) for img in pne_test]
test_imgs = np.array(cov_test_imgs+pne_test_imgs)
test_imgs_scaled = test_imgs.astype('float32')/255
test_labels = cov_test_num*['c']+pne_test_num*['p']

input_shape = (IMG_HEIGHT,IMG_WIDTH,train_imgs.shape[3])

In [5]:
# encode class labels as 0/1
le = LabelEncoder()
le.fit(train_labels)
train_labels_enc = le.transform(train_labels)
test_labels_enc = le.transform(test_labels)

In [6]:
# siamese network referenced from the following:
# https://github.com/keras-team/keras/blob/master/examples/mnist_siamese.py

def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 2
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    pred = y_pred.ravel() < 1
    return np.mean(pred == y_true)

def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 1, y_true.dtype)))
#     return K.mean(y_pred[1])

def create_pairs(x, digit_indices):
    '''Positive and negative pair creation.
    Alternates between positive and negative pairs.
    '''
    pairs = []
    labels = []
    n = min([len(digit_indices[d]) for d in range(num_classes)]) - 1
    for d in range(num_classes):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            pairs += [[x[z1], x[z2]]]
            inc = random.randrange(1, num_classes)
            dn = (d + inc) % num_classes
            z1, z2 = digit_indices[d][i], digit_indices[dn][random.randrange(0,n+1)]
            pairs += [[x[z1], x[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels)

def create_all_pairs(x, digit_indices):
    '''Positive and negative pair creation.
    Alternates between positive and negative pairs.
    '''
    pairs = []
    labels = []
    n = min([len(digit_indices[d]) for d in range(num_classes)])

    # create all positive pairs for each class (two classes: (N choose 2)*2=N^2-N)
    for d in range(num_classes):
        comb = combinations(range(0,n),2) # select indices in class
        for i in list(comb):
            z1, z2 = digit_indices[d][i[0]], digit_indices[d][i[1]]
            pairs += [[x[z1], x[z2]]]
            labels += [1]
    
    # create all possible negative pairs (two classes: N^2)
    comb = combinations(range(0,num_classes),2) # select two different classes
    for d in list(comb):
        for i in range(n):
            for j in range(n):
                z1, z2 = digit_indices[d[0]][i], digit_indices[d[1]][j]
                pairs += [[x[z1], x[z2]]]
                labels += [0]

    return np.array(pairs), np.array(labels)

def vggnet_base(input_shape):

    # train last 2 layers (top 3 layers not included)
    vggnet = vgg16.VGG16(include_top=False,weights='imagenet',input_shape=input_shape,pooling='avg')
    for layer in vggnet.layers[-5:]:
        layer.trainable=True
    for layer in vggnet.layers[:-5]:
        layer.trainable=False
    x = vggnet.output
    
#     imgIn = Input(shape=input_shape)
#     x = Flatten()(imgIn)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
#     x = Dropout(0.1)(x)
#     x = BatchNormalization()(x)
#     x = Dense(512, activation='relu')(x)
#     x = Dropout(0.1)(x)
#     x = Dense(1, activation='sigmoid')(x)
        
    return Model(vggnet.input,x)
#     return Model(imgIn,x)


In [7]:
# create positive and negative pairs
idx = [np.where(train_labels_enc==i)[0] for i in range(num_classes)]
tr_pairs, tr_y = create_all_pairs(train_imgs_scaled,idx)

idx = [np.where(test_labels_enc==i)[0] for i in range(num_classes)]
te_pairs, te_y = create_all_pairs(test_imgs_scaled,idx)

print('Training set original size: '+str(tr_pairs.shape[0]))
print('Testing set original size: '+str(te_pairs.shape[0]))

# use random subset of all pairs (might need to adjust to ensure class balance)
num_tr_pairs = 4096;
idx = random.sample(range(tr_pairs.shape[0]),num_tr_pairs)
tr_pairs = np.array([tr_pairs[x] for x in idx])
tr_y = np.array([tr_y[x] for x in idx])

num_te_pairs = 2048;
idx = random.sample(range(te_pairs.shape[0]),num_te_pairs)
te_pairs = np.array([te_pairs[x] for x in idx])
te_y = np.array([te_y[x] for x in idx])

print('Training set new size: '+str(tr_pairs.shape[0]))
print('Testing set new size: '+str(te_pairs.shape[0]))

# setup image augmentation on pairs of images using ImageDataGenerator, referenced from the following:
# https://github.com/keras-team/keras/issues/3386#issuecomment-237555199

def trainGenerator( X, I, Y):

    while True:
        # shuffled indices    
        idx = np.random.permutation( X.shape[0])
        # create image generator
        datagen = ImageDataGenerator(
                fill_mode='constant',
                cval=0,
                rescale=1./1,
                featurewise_center=False,  # set input mean to 0 over the dataset
                samplewise_center=False,  # set each sample mean to 0
                featurewise_std_normalization=False,  # divide inputs by std of the dataset
                samplewise_std_normalization=False,  # divide each input by its std
                zca_whitening=False,  # apply ZCA whitening
                rotation_range=5, #180,  # randomly rotate images in the range (degrees, 0 to 180)
                width_shift_range=0.05, #0.1,  # randomly shift images horizontally (fraction of total width)
                height_shift_range=0.05, #0.1,  # randomly shift images vertically (fraction of total height)
                horizontal_flip=True,  # randomly flip images
                vertical_flip=False)  # randomly flip images

        batches = datagen.flow( X[idx], Y[idx], batch_size=64, shuffle=False)
        idx0 = 0
        for batch in batches:
            idx1 = idx0 + batch[0].shape[0]

            yield [batch[0], I[ idx[ idx0:idx1 ] ]], batch[1]

            idx0 = idx1
            if idx1 >= X.shape[0]:
                break
                
def testGenerator( X, I, Y):

    while True:
        # suffled indices    
        idx = np.random.permutation( X.shape[0])
        # create image generator
        datagen = ImageDataGenerator(
                rescale=1./1)

        batches = datagen.flow( X[idx], Y[idx], batch_size=64, shuffle=False)
        idx0 = 0
        for batch in batches:
            idx1 = idx0 + batch[0].shape[0]

            yield [batch[0], I[ idx[ idx0:idx1 ] ]], batch[1]

            idx0 = idx1
            if idx1 >= X.shape[0]:
                break


Training set original size: 39060
Testing set original size: 3003
Training set new size: 4096
Testing set new size: 2048


In [8]:
# create siamese network with euclidean distance as final layer
base_network = vggnet_base(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclidean_distance,output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model = Model([input_a, input_b], distance)

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 128)          14796864    input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 1)            0           model_1[1][0]              

In [9]:
adm = optimizers.Adam(lr=0.0001)
model.compile(loss=contrastive_loss, optimizer=adm, metrics=[accuracy])

# use unedited training images
model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], tr_y,
          batch_size=128,
          epochs=10,
#           validation_data=([te_pairs[:, 0], te_pairs[:, 1]], te_y),
          shuffle=False)

# use augmented training images
# history = model.fit_generator(testGenerator(tr_pairs[:, 0],tr_pairs[:, 1],tr_y), 
#                               steps_per_epoch=1, 
#                               epochs=100,
#                               validation_data=testGenerator(te_pairs[:, 0],te_pairs[:, 1],te_y), 
#                               validation_steps=1, 
#                               verbose=1)

tr_y_pred = model.predict([tr_pairs[:, 0], tr_pairs[:, 1]])
tr_acc = compute_accuracy(tr_y, tr_y_pred)
te_y_pred = model.predict([te_pairs[:, 0], te_pairs[:, 1]])
te_acc = compute_accuracy(te_y, te_y_pred)

print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))

Epoch 1/10
4096/4096 [==============================] - 1376s 336ms/step - loss: 0.8227 - accuracy: 0.6758
Epoch 2/10
4096/4096 [==============================] - 1376s 336ms/step - loss: 0.1832 - accuracy: 0.9817
Epoch 3/10
4096/4096 [==============================] - 1376s 336ms/step - loss: 0.0891 - accuracy: 0.9985
Epoch 4/10
4096/4096 [==============================] - 1375s 336ms/step - loss: 0.0749 - accuracy: 0.9993
Epoch 5/10
4096/4096 [==============================] - 1378s 337ms/step - loss: 0.0653 - accuracy: 0.9998
Epoch 6/10
4096/4096 [==============================] - 1376s 336ms/step - loss: 0.0563 - accuracy: 1.0000
Epoch 7/10
4096/4096 [==============================] - 1373s 335ms/step - loss: 0.0503 - accuracy: 1.0000
Epoch 8/10
4096/4096 [==============================] - 1403s 342ms/step - loss: 0.0459 - accuracy: 1.0000
Epoch 9/10
4096/4096 [==============================] - 1467s 358ms/step - loss: 0.0416 - accuracy: 1.0000
Epoch 10/10
4096/4096 [==============

In [18]:
print(model.metrics_names)
print(np.mean(tr_y)) # show positive/negative balance
print(np.mean(te_y)) # show positive/negative balance

['loss', 'accuracy']
0.494873046875
0.48974609375
